<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga librerías</span>

In [1]:
import ultralytics
import sklearn
from ultralytics import YOLO
from ultralytics import settings
import os
import torch
import subprocess
import gdown
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import shutil
from sklearn.model_selection import train_test_split
import yaml
from threading import Thread
from queue import Queue
from PIL import Image
import re
from pathlib import Path
import cv2
import tempfile

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga rutas</span>

In [2]:
current_folder= os.getcwd() # Obtener ubicación actual
main_folder= os.path.abspath(os.path.join(current_folder, "../../../")) # Carpeta main del proyecto
main_staticos= os.path.abspath(os.path.join(main_folder))+'\\dist' # Carpeta main de los archivos estaticos
main_production= os.path.abspath(main_folder) +'\\src' # Carpeta main de los archivos listos para producción 
dir_prueba= os.path.abspath(main_folder) + "\\src\\assets_affectation"  # Carpeta de imagenes prueba para la tarea de predicción

training_structure="lote2_2" # Indica el nombre de la carpeta que tiene los archivos para el entrenamiento (image, json)
dir_train= os.path.abspath(main_staticos) + f'\\{training_structure}' # Establece la ruta completa del data para entrenamiento

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Función para instalar paquetes</span>

In [ ]:
# Función para la implementación de paquetes
def package_verification(package):
    try:
        output = os.popen('pip list').read()
        return f'{package}' in output
    except Exception as e:
        print(f'Error en la verificación: {e}')
        return False

def package_installation(package_name):
    if package_verification(package_name):
        print(f"El paquete '{package_name}' ya está instalado. ")
    else:
        print('Instalando el paquete... ')
        try:
            os.system(f'pip install {package_name}')
            print(f'libreria {package_name} instalada correctamente! ')
        except Exception as e:
            print(f'Error en la instalación de {package_name}: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de recursos para YOLO</span>

### Librería Ultralytics

In [ ]:
package_installation('ultralytics') # Instalación de la biblioteca Ultralytics
#ultralytics.checks()

### PyTorch

In [ ]:
#Función para determinar si tiene GPU Cuda para instalación de PYTorch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'CUDA: {torch.cuda.get_device_name(0)}')
else:
    try:
        command = 'wmic path win32_videocontroller get caption'
        device = subprocess.check_output(command, shell=True, universal_newlines=True)
        print(f'Tarjeta Grafica: {device.strip()}.\n \nAdvertencia: Debe utilizar la CPU para instalación de PyTorch! ')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar el comando: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Dirección para instalar mediante el comando la versión de PyTorch en función de los requerimientos computacionales: </span>[Versión PyTorch](https://pytorch.org/get-started/locally/)

In [ ]:
command_pytorch = 'pip3 install torch torchvision torchaudio' # Comando de instalación según la version personalizada a sus requerimientos computacionales # idear solucion mas amena al cliente
substrings = command_pytorch.split(" ")
try:
    subprocess.check_call(substrings)
except subprocess.CalledProcessError as e:
    print(f'Error al instalar la version de PyTorch: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de herramientas de etiquetado</span>

In [2]:
try:
    package_installation('labelme') # Instalación de la herramienta de etiquetado LabelMe
except NameError:
    print('La función destinada a la instalación de dependencias no está activa. Por favor, ejecute primero la celda correspondiente. ')

La función destinada a la instalación de dependencias no está activa. Por favor, ejecute primero la celda correspondiente. 


In [4]:
try:
    subprocess.check_call(["labelme"]) # Permite abrir la herramienta LabelMe
except Exception as e:
    print(f'No fue posible abrir labelme: {e}')

No fue posible abrir labelme: name 'subprocess' is not defined


In [3]:
try:
    package_installation('labelme2yolo') # Instalación del convertidor para transformar el formato JSON de LabelMe al formato de texto exigido por los modelos YOLO de detección de objetos.
except NameError: 
    print('La función destinada a la instalación de dependencias no está activa. Por favor, ejecute primero la celda correspondiente. ')

La función destinada a la instalación de dependencias no está activa. Por favor, ejecute primero la celda correspondiente. 


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de rutas de entrenamiento | Movimiento de datos globales etiquetados</span>

In [14]:
# Función para separar la carpeta global de todos los datos en un 80% entrenamiento y 20% validación. Utilice esta función solo si no ha ordenado los datos en una carpeta para entrenamiento y otra para validación.

def file_separation(orig_folder):
    file_list = os.listdir(orig_folder)
    train, val = train_test_split(file_list, test_size=0.2, random_state=100)
    if not os.path.exists("train"):
        train_path = os.path.join(orig_folder, "train")
        os.makedirs(train_path, exist_ok=True)
    move_files(train, orig_folder, train_path)
    
    if not os.path.exists("val"):
        val_path = os.path.join(orig_folder, "val")
        os.makedirs(val_path, exist_ok=True)
    move_files(val, orig_folder, val_path)

def move_files(file_list, orig_folder, dest_folder):
    for file in file_list:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            orig_image_path = os.path.join(orig_folder, file)
            json_name = os.path.splitext(file)[0] + ".json"
            
            orig_json_path = os.path.join(orig_folder, json_name)
            dest_image_path = os.path.join(dest_folder, file)
            dest_json_path = os.path.join(dest_folder, json_name)
            
            shutil.copy(orig_image_path, dest_image_path)
            shutil.copy(orig_json_path, dest_json_path)

file_separation(dir_train)

Cantidad de archivos de entrenamiento: 399
Cantidad de archivos de validación: 98


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Conversión a formato YOLO</span>

In [61]:
# Función para convertir la carpeta de entrenamiento y validación en formato YOLO

def convert_to_yolo_format(training_directory, validation_directory):
    try:
        subprocess.check_call(["labelme2yolo", "--json_dir", training_directory])
        print(f'Conversión completada para la carpeta {training_directory}')
        subprocess.check_call(["labelme2yolo", "--json_dir", validation_directory])
        print(f'Conversión completada para la carpeta {validation_directory}')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar función labelme2yolo: {e}')

train_path= os.path.abspath(dir_train) + "\\train"
val_path= os.path.abspath(dir_train) + "\\val"

convert_to_yolo_format(train_path, val_path)

Conversión completada para la carpeta c:\Users\matrix\pruebayolo\proyecto_yolo\dist\lote2_2\train
Conversión completada para la carpeta c:\Users\matrix\pruebayolo\proyecto_yolo\dist\lote2_2\val


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Conversión a formato YOLO | Configuración del dataset</span>

In [16]:
# Organización de la estructura del repositorio YOLODataset (resultado de la conversión .json a formato YOLO) creado por defecto

def reorganization(path):
    try:
        new_path = os.path.abspath(os.path.join(path, "YOLODataset"))
        main_folders = os.listdir(new_path)
        for folder_instance in main_folders:
            secondary_folder_path = os.path.join(new_path, folder_instance)
            if os.path.isdir(secondary_folder_path):
                secondary_folders = os.listdir(secondary_folder_path)
                for secondary_folder in secondary_folders:
                    subfolder_secondary_path = os.path.join(secondary_folder_path, secondary_folder)
                    try:
                        if os.listdir(subfolder_secondary_path) != []:
                            for file in os.listdir(subfolder_secondary_path):
                                orig_path = os.path.join(subfolder_secondary_path, file)
                                dest_path = os.path.abspath(os.path.join(subfolder_secondary_path, "../"))
                                new_dest_path = os.path.join(dest_path, file)
                                shutil.move(orig_path, new_dest_path)
                            os.rmdir(subfolder_secondary_path)
                        else:
                            os.rmdir(subfolder_secondary_path)
                    except NotADirectoryError as e:
                        print(f'Error: the element is not a folder: {e}')
        return new_path
    except FileNotFoundError as e:
        print(f'Error: {e}')

yolo_train_data= reorganization(train_path)
yolo_val_data= reorganization(val_path)    

In [17]:
# Función para mover las carpetas yolo 

def acomodar_rutas_dataset(main_staticos, dataset_train, dataset_val):
    try:
        if not os.path.exists("data_affectation"):
            try: 
                paths_segmentation= os.path.join(main_staticos, "data_affectation")
                os.makedirs(paths_segmentation, exist_ok=True)
                if os.listdir(paths_segmentation) == []:
                    shutil.move(dataset_train, os.path.join(paths_segmentation, "YOLODataset_train"))
                    shutil.move(dataset_val, os.path.join(paths_segmentation, "YOLODataset_val"))
                    print(f'Carpetas movidas exitosamente a {paths_segmentation} ')
                else:
                    print(f'Las carpetas ya se encuentran en {paths_segmentation} ')
            except OSError as e:
                print(f'Error al intentar crear la carpeta en {paths_segmentation}: {e}')
        return paths_segmentation  
    except TypeError as e:
        print(f'Tipo de dato incorrecto para el argumento: {e}')

segmentation_routes= acomodar_rutas_dataset(main_staticos, yolo_train_data, yolo_val_data)

Carpetas movidas exitosamente a c:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_afectacion 


In [18]:
# Set de funciones para ordenar la estructura de los dataset individuales.

def arrange_dataset_paths(train, val):
    destination = os.path.abspath(os.path.join(train, "../../"))
    shutil.move(train, destination)
    if "dataset.yaml" in os.listdir(os.path.join(val, "../")):
        os.remove(val)

# Función para reescribir el dataset
def rewrite_dataset(dir_train_converted, combined_list):
    with open(dir_train_converted, 'w') as new_dataset:
        new_dataset.write(
            f'train: {os.path.abspath(os.path.join(dir_train_converted, "../"))}\n'
            f'val: {os.path.abspath(os.path.join(dir_train_converted, "../../"))}\\YOLODataset_val\n'
            'test: \n'
            f'nc: {len(combined_list)}\n'
            f'names: {combined_list}\n'
        )
        
# Función para combinar los dos dataset generados
def total_labels(list1, list2, update_dataset_train):
    set1=set(list1)
    set2=set(list2)
    union_without_repeating = set1.union(set2)
    combined_list = list(union_without_repeating)
    rewrite_dataset(update_dataset_train, combined_list)

# Función para extraer los nombres de las etiquetas de los dos dataset separados
def extract_labels(main_route):
    try:
        route_dataset_train= os.path.abspath(main_route) + "\\YOLODataset_train\\dataset.yaml"
        route_dataset_val= os.path.abspath(main_route) + "\\YOLODataset_val\\dataset.yaml"
        
        with open(route_dataset_train, 'r') as file_train:
            data_train = yaml.safe_load(file_train)
            labels_train = data_train['names']
        with open(route_dataset_val,'r') as file_val:
            data_val= yaml.safe_load(file_val)
            labels_val= data_val['names']
        total_labels(labels_train, labels_val, route_dataset_train)
        
        return route_dataset_train, route_dataset_val
    except (TypeError, FileNotFoundError) as e:
        print(f'Error: {e}')

try:
    train, val= extract_labels(segmentation_routes)
    arrange_dataset_paths(train, val)
except TypeError as e:
    print(f'Error: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Ubicación de la Carpeta Preparada para el Entrenamiento</span>

In [4]:
# Buscar la carpeta que contiene la configuración completa para el entrenamiento
data_train= os.path.abspath(main_staticos) + "\\data_affectation\\dataset.yaml"

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de las carpetas predeterminadas de YOLO</span>

In [20]:
# Modificación de las rutas de las carpetas predeterminadas de ultralytics YOLO

def set_config_ultralytics(main):
    settings.update({'datasets_dir': f'{main}\\dist', 'weights_dir': f'{main}\\src', 'runs_dir': f'{main}\\src\\runs'})
set_config_ultralytics(main_folder)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Selección versión de modelo YOLOv8</span>

### Tabla de variantes para la segmentación de objetos
| Modelo      | Nombres de archivo                                                         | Identificador |
|-------------|-----------------------------------------------------                       |------------|
| YOLOv8      | yolov8n-seg.pt yolov8s-seg.pt yolov8m-seg.pt yolov8l-seg.pt yolov8x-seg.pt | n, s, m, l, x |


### Instalación del paquete para la gestión de descarga de los modelos

In [ ]:
package_installation('gdown') # Librería que permitirá descargar las versiones de YOLOv8

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Modelo original</span>

In [4]:
def get_model_version():
    available_versions= ['n', 's', 'm', 'l', 'x'] # yolov8n.pt , yolov8s.pt , yolov8m.pt , yolov8l.pt , yolov8x.pt
    control= True
    while control:
        version_selection= input("Ingrese la versión del modelo que desea utilizar, entre las disponibles están: n, s, m, l, x. Advertencia: Si no desea instalar ninguna ingrese la palabra 'salir' ").lower() 
        if version_selection == "salir":
            id_version=None
            control=False
        elif version_selection in available_versions:
            id_version= f'yolov8{version_selection}-seg.pt' 
        else:
            print('Version de modelo inexistente, ingrese nuevamente! ')
        return id_version
     
def model_installation(url, version_name):
    
    path_save = os.path.abspath(main_folder) + "\\dist\\model_version\\" + version_name
    if os.path.exists(path_save):
        print(f'Se está utilizando la versión "{version_name}" del modelo. ')
    else:
        try:
            url_installation = f'{url}/{version_name}'
            print(url_installation)
            gdown.download(url_installation, path_save, quiet=False)
            print('Versión instalada correctamente!')
        except (NameError, UnboundLocalError) as e:
            print(f'Error en la instalación de la versión del modelo: {e}')
    return path_save

model_url= 'https://github.com/ultralytics/assets/releases/download/v0.0.0' 
model_version= get_model_version()

if model_version is not None:
    model_orig= model_installation(model_url, model_version) 
else:
    print('Ha salido con exito! ')

Se está utilizando la versión "yolov8n-seg.pt" del modelo. 


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Modelo pre-entrenado</span>

In [5]:
# Función para obtener el último entrenamiento

def get_last_training(dir_src, det_type):
    try:
        yolo_model_folders= os.listdir(dir_src)
        for folder_name in yolo_model_folders:
            if folder_name == det_type:
                base_path = os.path.abspath(os.path.join(dir_src, folder_name))
                files= os.listdir(base_path)
                last_experiment = sorted(files)[-1]
                model= os.path.join(base_path, last_experiment, "weights", "best.pt")
                print(f'Los pesos del experimento que se van a utilizar son {model}')
                return model
        else:
            print(f'No existe ningún experimento de entrenamiento en la carpeta {dir_src}')
            return None
    except FileNotFoundError as e:
        print(f'Error: {e}')
        
# Llamado para obtener el último entrenamiento del modo segmentación
try:
    version_afectation= "srcv5" # Nombre de la carpeta donde se encuentra el modelo pre-entrenado a usar
    detection_type_afectation= "segmentation" # Establece el modo de detecciones de Yolo (deteccion o segmentación)
    dir_model_version_afectation=os.path.abspath(os.path.join(main_production, version_afectation)+ f"\\model_{detection_type_afectation}") 
    model_afectation= get_last_training(dir_model_version_afectation, detection_type_afectation) 
except NameError:
    print('Las rutas no se encuentran activas. ')

Los pesos del experimento que se van a utilizar son c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation\weights\best.pt


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">ENTRENAMIENTO</span>


In [5]:
PROJECT='segmentation' # Permite asignar un nombre al directorio de inicio que contendrá los experimentos de segmentación de objetos, y debe estar entre comillas; se recomienda no utilsizar espacios en el nombre.
NAME='training_affectation' # El nombre del experimento entrenamiento para segmentación de instancias debe ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
TASK='segment' # Define la tarea principal que desea realizar con el modelo YOLO v8; en este caso, la segmentación implica el uso de máscaras para identificar objetos individuales en una imagen y segmentarlos del resto de la imagen.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada. Puede especificarlo como un número entero, como imgsz a 640 para obtener un cuadrado perfecto, o como una tupla, como imgsz=(640,480) para establecer dimensiones específicas de ancho y alto. Se recomienda ajustar este valor según el tamaño del objeto que desea segmentar. Para la segmentación de objetos pequeños, se recomienda aumentar el valor a más de 640 píxeles para obtener una resolución más alta.
DATA= data_train #  Le permite indicar la ruta al archivo que contiene los metadatos que utilizara el modelo segmentación de objetos y su configuración en formato YAML. Si especifica el valor data=None, el conjunto de datos coco128-seg.yaml se utiliza de forma predeterminada; de lo contrario, escriba la ruta al archivo YAML entre comillas utilizando barras diagonales (/) en lugar de barras invertidas (\).
EPOCHS=30 # Establezca el número de épocas del modelo YOLO v8 en la tarea de segmentación. Este valor representa el número total de iteraciones en todo el conjunto de datos de entrenamiento. Se recomienda experimentar con este parámetro dependiendo de la cantidad de imágenes disponibles. Si tiene un conjunto de datos grande, considere aumentar este valor por encima de 30 para obtener mejores resultados. Por otro lado, establecer epochs=None hará que el modelo continúe entrenándose hasta que la pérdida de validación deje de mejorar.
BATCH=-1 # Define la cantidad de imágenes procesadas simultáneamente en una iteración del modelo YOLO v8 en la segmentación. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
OPTIMIZER='auto' # Define el algoritmo de optimización para el modelo de segmentación YOLO. Su elección ajusta los pesos del modelo durante el entrenamiento y es crucial para la velocidad y rendimiento. Puede tomar valores como 'SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp' y 'auto', este último selecciona automáticamente el optimizador más adecuado a la tarea segmentación de objetos.
WORKERS=1 # Especifica la cantidad de subprocesos o núcleos utilizados para cargar datos en el modelo segmentación YOLO. Se recomienda ajustar la cantidad de subprocesos a la cantidad de núcleos de CPU disponibles en el sistema.
DEVICE= 'cpu' # Especifica el dispositivo de ejecución para la versión del modelo yolov8 en la operación de segmentación. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante el entrenamiento de segmentación de objetos. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
SAVE=False # Cuando se establece en True, el modelo guarda puntos de control periódicamente durante el entrenamiento de segmentación. Se recomienda tener cuidado al establecer este valor en Verdadero, ya que está relacionado con el hiperparámetro SAVE_PERIOD; si establece el valor del hiperparametro a False, la función Save_Period se desactivará.
SAVE_PERIOD=-1 # Se utiliza para especificar con qué frecuencia se guardan los puntos de control durante el entrenamiento de segmentación. Si se establece en un valor mayor que 0, el modelo guardará puntos de control cada número especificado de épocas. Sin embargo, si save_period se establece en -1, significa que la función esta deshabilitada.
PATIENCE=30 # Representa el número esperado de épocas durante el entrenamiento de segmentación. Si no se observa mejora en el conjunto de validación dentro de un período específico, se detiene el proceso. Esta técnica de parada temprana se utiliza para evitar el sobreajuste del modelo. Se recomienda ajustar este hiperparámetro en función de la duración esperada del entrenamiento.
VERBOSE=False # Se utiliza para controlar el número de impresiones durante la ejecución del entrenamiento de segmentación. Para suprimir la salida de información básica únicamente, debe establecer el valor del hiperparámetro en False, pero si desea una salida de progreso más detallada, establezca el valor en True.
RECT= False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de entrenamiento en la segmentación de instancias. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente lote.
COS_LR=False # Reemplaza el decaimiento escalonado predeterminado de YOLOv8, que reduce la tasa de aprendizaje en ciertas épocas, con el decaimiento escalonado cos_lr. Este ajusta la tasa de aprendizaje según las épocas restantes y la tasa de aprendizaje inicial, proporcionando una disminución más suave. Establezca este hiperparámetro en True para una reducción gradual de la tasa de aprendizaje, de lo contrario establezca en False.
FRACTION= 1.0 # Controla la fracción del conjunto de datos que se utilizara para el entrenamiento de segmentación. Debes establecer este parámetro entre 0.0 y 1.0. Por defecto, cuando es 1.0, se emplea el 100% de las imágenes disponibles en el conjunto de datos.
EXIST_OK=False # Controla la sobreescritura de un experimento de segmentación existente. Cuando se establece en False, el sistema no sobrescribirá, en su lugar devolverá una ruta incrementada. Esto es útil para prevenir la sobreescritura accidental de experimentos anteriores. Para activar la función, asigna el valor True.
OVERLAP_MASK=True # Determina si las máscaras que representan áreas de interés en la imagen deben superponerse. Establecer el valor del hiperparametro a True permite que estas máscaras compartan áreas, lo que significa que los límites de los objetos en la imagen puede coincidir parcialmente. Esta opción ahorra memoria, acelera el entrenamiento, y es especialmente útil para grandes conjuntos de datos o modelos complejos.
MASK_RATIO=1 # Configure la reducción de muestreo para la máscara de segmentación. Se sugiere fijar el valor en el rango de 1 para entrenamiento en resolución nativa, ideal para aplicaciones que demandan alta precisión, hasta 4 para una reducción en un factor de 4, acelerando el entrenamiento con una precisión aceptable y ahorrando memoria. Este hiperparámetro solo acepta valores enteros.

def train(model_segmentation):
    try:
        model_segmentation.train(project=PROJECT,name=NAME, task=TASK, data=DATA, imgsz=IMGSZ, epochs=EPOCHS, batch=BATCH, workers=WORKERS, device=DEVICE, plots=PLOTS, verbose=VERBOSE, rect=RECT, cos_lr=COS_LR, optimizer=OPTIMIZER, patience=PATIENCE, exist_ok=EXIST_OK, overlap_mask=OVERLAP_MASK, mask_ratio=MASK_RATIO, fraction=FRACTION)
    except Exception as e:
        print(f'Error: {e}')

# --Inicio Menú
while True:
    try:
        option_train = input('Por favor, indique el tipo de modelo que desea utilizar: "original" o "entrenado". Si desea cancelar el proceso, indique "salir": ').lower()
        if option_train == "salir":
            break
        elif option_train == "original":
            train(YOLO(model_orig))
            break  # Sale del bucle si la entrada es válida
        elif option_train == "entrenado":
            train(YOLO(model_afectation)) 
            break  # Sale del bucle si la entrada es válida
        else:
            print('Character not valid, try again. ')
    except Exception as e:
        print(f'Error when performing the training task: {e}')
# --Finalización Menú

New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
engine\trainer: task=segment, mode=train, model=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\model_version\yolov8n-seg.pt, data=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_affectation\dataset.yaml, epochs=30, patience=30, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=1, project=segmentation, name=training_affectation, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=1, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False,

train: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_affectation\YOLODataset_train\labels.cache... 199 images, 0 backgrounds, 0 corrupt: 100%|██████████| 199/199 [00:00<?, ?it/s]
val: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_affectation\YOLODataset_val\labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]

Plotting labels to segmentation\training_affectation\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to segmentation\training_affectation
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/30         0G      2.114      4.487      3.576      1.754        100        640: 100%|██████████| 13/13 [02:12<00:00, 10.15s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:11<00:00,  5.65s/it]

...

      30/30         0G      1.524       2.52      1.736      1.307         35        640: 100%|██████████| 13/13 [01:50<00:00,  8.51s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:10<00:00,  5.48s/it]

                   all         49        235      0.282      0.289      0.205     0.0776      0.181      0.175      0.086     0.0282



30 epochs completed in 1.083 hours.
Optimizer stripped from segmentation\training_affectation\weights\last.pt, 6.8MB
Optimizer stripped from segmentation\training_affectation\weights\best.pt, 6.8MB

Validating segmentation\training_affectation\weights\best.pt...
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:10<00:00,  5.20s/it]


                   all         49        235      0.226      0.303      0.186     0.0919      0.252       0.13      0.108     0.0408
Speed: 2.1ms preprocess, 111.6ms inference, 0.0ms loss, 8.3ms postprocess per image
Results saved to segmentation\training_affectation


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">Configuración para la exportación</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 18px;">A continuación se muestra una tabla de referencia para exportar un modelo YOLOv8 entrenado en la tarea segmentación de instancias. Tenga en cuenta que configurar el parámetro de `format` es fundamental para el proceso de exportación. Antes de continuar, asegúrese de verificar y ajustar estos valores a sus requisitos específicos: </span>

| Formatos                                                             | Asignación | Extensión                     | Hiperparámetros                                            | Descripción                                        |
|--------------------------------------------------------------------|-------------------|---------------------------|-----------------------------------------------------|----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | -                                                   | Modelo en formato PyTorch                           |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | "torchscript"     | `yolov8n.torchscript`     | `imgsz`, `optimize`                                 | Simplifica la implementación de modelos PyTorch en entornos de producción y aplicaciones eficientes, mejorando la portabilidad y el rendimiento al permitir ejecutar una representación intermedia en entornos sin Python.                       |
| [ONNX](https://onnx.ai/)                                           | "onnx"            | `yolov8n.onnx`            | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     | Desarrollado para promover la interoperabilidad, la optimización del hardware y la colaboración entre comunidades, al tiempo que responde a la necesidad de portabilidad de los modelos entre distintos marcos y herramientas de aprendizaje automático.                              |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | "openvino"        | `yolov8n_openvino_model/` | `imgsz`, `half`, `int8`                             | Elaborado para promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware Intel.                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | "engine"          | `yolov8n.engine`          | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` | Permite promover la interoperabilidad, la optimización del hardware y la implantación eficiente de modelos en distintos marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de NVIDIA.                     |
| [CoreML](https://github.com/apple/coremltools)                     | "coreml"          | `yolov8n.mlpackage`       | `imgsz`, `half`, `int8`, `nms`                      | Posibilita promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de Apple.                            |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | "saved_model"     | `yolov8n_saved_model/`    | `imgsz`, `keras`, `int8`                            | Empleado para guardar, compartir y desplegar modelos entrenados con TensorFlow. Versátil y facilita el despliegue en diversas plataformas como servidores, dispositivos móviles, embebidos y navegadores.                     |
| [TF Lite](https://www.tensorflow.org/lite)                         | "tflite"          | `yolov8n.tflite`          | `imgsz`, `half`, `int8`                             | Diseñado para el aprendizaje automático en dispositivos, TF Lite aborda restricciones clave como latencia, privacidad, conectividad, tamaño y consumo de energía. Es esencial para desplegar modelos en dispositivos móviles e integrados, ofreciendo una solución ligera y eficiente.                           |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | "edgetpu"         | `yolov8n_edgetpu.tflite`  | `imgsz`                                             | Utilizado para desplegar modelos de aprendizaje automático en el Edge TPU de TensorFlow. El Edge TPU es un pequeño ASIC (Circuito Integrado Específico de Aplicación) diseñado por Google para ofrecer inferencias de aprendizaje automático de alto rendimiento en dispositivos de bajo consumo.                       |
| [TF.js](https://www.tensorflow.org/js)                             | "tfjs"            | `yolov8n_web_model/`      | `imgsz`                                             | Facilita el despliegue de modelos de aprendizaje automático en navegadores web y Node.js, destacando la portabilidad y la facilidad de uso.                    |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | "paddle"          | `yolov8n_paddle_model/`   | `imgsz`                                             | Utilizado para desplegar modelos en PaddlePaddle, una plataforma de aprendizaje profundo de código abierto, paralela y distribuida que tiene su origen en la práctica industrial.                      |
| [ncnn](https://github.com/Tencent/ncnn)                            | "ncnn"            | `yolov8n_ncnn_model/`     | `imgsz`, `half`                                     | Formato optimizado para plataformas móviles, ofreciendo alto rendimiento. Puede incluir una estructura de archivo de modelo con información sobre capas, blobs de entrada y salida, y otros parámetros.                              |


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">EXPORTACIÓN</span>


In [9]:
FORMAT='onnx' # Seleccione el formato de exportación del modelo segmentación, empleando la tabla previamente proporcionada; en la columna "Asignación" se indican las opciones para ajustar este valor.
INT8=False # Establezca este parámetro en True al utilizar la CPU y en False en caso contrario. La cuantificación a INT8 mejora la eficiencia del modelo segmentación en cuanto a memoria y velocidad de inferencia, especialmente en hardware que admite esta precisión.
HALF=False # Configúrelo en True cuando use la GPU; en caso contrario, False. La cuantificación a FP16 mejora la eficiencia de la memoria del modelo segmentación y la velocidad de inferencia, especialmente en hardware que admite precisión de punto flotante de 16 bits.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada para la exportación del modelo de segmentación. Puede especificarlo como un número entero, por ejemplo, 640 para un cuadrado perfecto, o como una tupla, por ejemplo, (640, 480) para dimensiones específicas de ancho y alto. Las imágenes que ingreses al modelo después de la exportación deben tener las mismas dimensiones específicas que has configurado para adaptarse a los requisitos del escenario de despliegue.
OPTIMIZE=False # Controla la optimización en modelos de segmentación de instancias a TorchScript para su implementación móvil. Es importante destacar que esta función puede resultar en un aumento significativo en el tamaño del modelo exportado, lo cual puede no ser ideal para aplicaciones móviles. Se configura con True para activar y False para desactivar.
DYNAMIC=False # Controla la habilitación de ejes dinámicos en modelos de segmentación, lo cual es particularmente útil para gestionar tamaños de lote variables. Esta característica funciona bien en escenarios donde el tamaño del lote puede cambiar durante la inferencia, como aplicaciones en tiempo real o de transmisión por secuencias. Los valores aceptados son Verdadero para habilitar la función y Falso para deshabilitarla.
SIMPLIFY=False # En la exportación de modelos de segmentación a ONNX|TensorRT, este hiperparámetro personaliza la complejidad del modelo, optimizando, eliminando capas redundantes y reduciendo la precisión de los parámetros. Se activa con True y se desactiva con False.
OPSET=None # Especifica la versión del conjunto de operadores en ONNX al exportar el modelo segmentación desde marcos como PyTorch o TensorFlow. Si se deja en "None", ONNX utilizará automáticamente la versión más reciente disponible; para una versión específica, asigne el número entre comillas, por ejemplo, "11".
WORKSPACE=4.0 # En la exportación de modelos de segmentación a TensorRT, establece el tamaño del espacio de trabajo en GB asignado para optimizar y preparar el modelo de red neuronal. Este espacio se utiliza durante el proceso de construcción del motor para lograr una ejecución eficiente en hardware GPU mediante la biblioteca TensorRT.
NMS=False # En la exportación de modelos de segmentación a CoreML, controla la inclusión de la Supresión No Máxima (NMS) en el modelo para eliminar cuadros delimitadores redundantes en la segmentación de instancias y mejorar la precisión de las predicciones. Establecer 'NMS' en 'False' ignora NMS en los modelos CoreML exportados.  Este ajuste, configurable durante la exportación del modelo YOLO, lo que permite a los usuarios optimizar la implementación del modelo en una variedad de plataformas y dispositivos.
KERAS= False # En la exportación de modelos de segmentación a TF SavedModel y TF Lite, permite optimizar el despliegue en diversas plataformas y dispositivos. Incluye también el formato del archivo, el dispositivo de ejecución y la posibilidad de manejar múltiples etiquetas por caja. Establezca el valor del hiperparámetro en True si está familiarizado con Keras; de lo contrario, en False para excluir su uso en la exportación.

def export(model_export):
    try:
        model_export=YOLO(model_export)
        model_export.export(format=FORMAT, imgsz=IMGSZ, dynamic=DYNAMIC, opset=OPSET, workspace=WORKSPACE, nms=NMS, keras=KERAS, int8=INT8, half=HALF, optimize=OPTIMIZE, simplify=SIMPLIFY)
    except Exception as e: 
        print(f'Error in the export of the model: {e}')

export(model_afectation)

Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from 'c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation2\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 38, 8400), (1, 32, 160, 160)) (6.5 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 1.6s, saved as 'c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation2\weights\best.onnx' (12.6 MB)

Export complete (3.9s)
Results saved to C:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation2\weights
Predict:         yolo predict task=segment model=c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation2\weights\best.onnx imgsz=640  
Va

In [16]:
# Opción valor por defecto flexible
class Exportacion:
    def __init__(self, model, format, imgsz=640, **kwargs): 
        self.model = model # dirección del modelo a exportar
        self.format = format # nombre del formato de exportación seleccionado 
        self.imgsz = imgsz # tamaño de imagen global
        self.hyperparameter = kwargs # diccionario con el valor de hiperparámetros a modificar
        self.formats_config = { # Configuración establecida por defecto, en la forma -> Formato : {hiperparámetro = valor}
            "torchscript": {'optimize': False},
            "onnx": {'half': False, 'dynamic': False, 'simplify': False, 'opset': None},
            "openvino": {'half': False, 'int8': False}, # requirements: Ultralytics requirement ['openvino>=2024.0.0'] 
            "engine": {'half': False, 'dynamic': False, 'simplify': False, 'workspace': 4.0}, # Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.
            "coreml": {'half': False, 'int8': False, 'nms': False}, # requirements: Ultralytics requirement ['coremltools>=7.0'] 
            "saved_model": {'keras': False, 'int8': False}, # requirements: Ultralytics requirements ['tf_keras', 'onnx2tf>1.17.5,<=1.22.3', 'sng4onnx>=1.0.1', 'onnxslim==0.1.28', 'onnx_graphsurgeon>=0.3.26', 'tflite_support'] 
            "tflite": {'half': False, 'int8': False}, # requirements: Ultralytics requirements ['tf_keras', 'onnx2tf>1.17.5,<=1.22.3', 'sng4onnx>=1.0.1', 'onnxslim==0.1.28', 'onnx_graphsurgeon>=0.3.26', 'tflite_support'] 
            "edgetpu": {}, # Edge TPU export only supported on Linux. See https://coral.ai/docs/edgetpu/compiler
            "tfjs": {}, # requirements: Ultralytics requirements ['tf_keras', 'onnx2tf>1.17.5,<=1.22.3', 'sng4onnx>=1.0.1', 'onnxslim==0.1.28', 'onnx_graphsurgeon>=0.3.26', 'tflite_support'] 
            "paddle": {}, # requirements: Ultralytics requirements ['paddlepaddle', 'x2paddle'] 
            "ncnn": {'half': False} # requirements: Ultralytics requirement ['ncnn'] 
        }
        
        
    def remove_parameters(self):
        # Elimina los parámetros que no son compatibles con el formato de exportación seleccionado
        keys_to_remove = [param for param in list(self.hyperparameter.keys()) if param not in self.formats_config]
        print(keys_to_remove)
           
        for invalid_parameter in keys_to_remove:
            del self.hyperparameter[invalid_parameter]
        
        self.apply_defaults()
            
    def apply_defaults(self):
    # Aplica los valores por defecto a los hiperparámetros que no han sido modificados por el usuario
        for param, default in self.formats_config[self.format].items():
            if param not in self.hyperparameter:  # Verifica si el usuario modificó el parámetro de manera manual
                self.hyperparameter[param] = default  # Asigna un valor por defecto al parámetro si no fue modificado
           
    def export_model(self):
        print(f'Exporting to the format: {self.format}\n Configuration included: imgsz={self.imgsz}, {", ".join([f"{k}={v}" for k, v in self.hyperparameter.items()])}')
        try:
            model_export = YOLO(self.model) 
            print('Successful export: ')
            model_export.export(format=self.format, imgsz=self.imgsz, **self.hyperparameter)
        except Exception as e: 
            print(f'Error in the export of the model: {e}')

# Menú
def export_format():
    formats= {'1': 'torchscript', '2': 'onnx', '3': 'openvino', '4': 'engine', '5': 'coreml', '6': 'saved_model', '7': 'tflite', '8': 'edgetpu', '9': 'tfjs', '10': 'paddle', '11': 'ncnn'} # Se establecen los formatos a los que se puede exportar los modelos YOLO
    while True:
        format_selection = input(f'Seleccione el número del formato a exportar el modelo: {", ".join([f"{identificador}: {valor}" for identificador, valor in formats.items()])}') 
        if format_selection in formats: # Verifica que el identificador coincida con un formato
            object_= Exportacion(model_afectation, formats[format_selection], half=True) # Crea una instancia de la clase, la cual recibe el modelo y el formato a exportar, además, de manera opcional se puede modificar el valor de los parámetros 
            object_.remove_parameters() 
            object_.export_model() 
            break 
        else: 
            print('Unsupported format! ')
              
export_format()

['half']
Exporting to the format: onnx
 Configuration included: imgsz=640, half=False, dynamic=False, simplify=False, opset=None
Successful export: 
Ultralytics YOLOv8.2.31  Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from 'c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 38, 8400), (1, 32, 160, 160)) (6.5 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success  1.5s, saved as 'c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation\weights\best.onnx' (12.6 MB)

Export complete (4.0s)
Results saved to C:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_segmentation\segmentation\training_affectation\weights
Predict:         yolo predict task=segment 

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">VALIDACIÓN</span>


In [10]:
NAME='validation_affectation_srcv5' # El nombre del experimento de validación para segmentación de instancias deben ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
DATA= data_train # Permite indicar la ruta al archivo que contiene los metadatos necesarios para el proceso de validación, la ruta debe ser proporcionada entre comillas.
SAVE_JSON=True # Si se configura como True, habilita la funcionalidad de guardar los resultados obtenidos de manera detallada del proceso de validación en un formato estructurado JSON.
IMGSZ=640 # Establece las dimensiones en píxeles de la imagen de entrada para la validación del modelo de segmentación. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda usar el mismo valor que utilizo durante el entrenamiento del modelo.
BATCH=16 # Define la cantidad de imágenes procesadas simultáneamente en una iteración para la validación de segmentos. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
SAVE_HYBRID=True # Activa la función con True para guardar una versión híbrida de la etiqueta, incluyendo la original y predicciones adicionales. Útil para el análisis detallado del rendimiento del modelo segmentación durante la validación; establezca en False para mostrar solo las predicciones.
CONF=0.5 # Establece el umbral de confianza para la validación de clases en la tarea de segmentación. Se recomienda un valor entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión. 
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara segmentar en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
DEVICE='CPU' # Especifica el dispositivo de ejecución para la prueba de validación en la operación de segmentación. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante la validación en la segmentación de instancias. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
RECT=False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de validación en la segmentación de instancias. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente.
IOU=0.6 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,6. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren el mismo segmento. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.

def val(model_val):
    try:
        model_val.val(name=NAME, data=DATA, save_json=SAVE_JSON, imgsz=IMGSZ, batch=BATCH, save_hybrid=SAVE_HYBRID, conf=CONF, max_det=MAX_DET, device=DEVICE, plots=PLOTS, rect=RECT, iou=IOU)
    except Exception as e:
        print(f'Error: {e}')

# --Inicio Menú
while True:
    try:
        option_val = input('Qué tipo de modelo desea utilizar, indique "original" o "entrenado": ').lower()
        if option_val == "salir":
            break
        if option_val == "original":
            val(YOLO(model_orig))
            break  # Sale del bucle si la entrada es válida
        elif option_val == "entrenado":
            val(YOLO(model_afectation))
            break  # Sale del bucle si la entrada es válida
        else:
            print('Character not valid, try again. ')
    except Exception as e:
        print(f'Error when performing the validation task: {e}')
# --Finalización Menú

Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_affectation\YOLODataset_val\labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:08<00:00,  2.08s/it]


                   all         49        235          1      0.865      0.929      0.929       0.01     0.0109    0.00502      0.001
            abolladura         49         46          1          1      0.995      0.995     0.0201     0.0217       0.01    0.00201
                 rayon         49        189          1       0.73      0.863      0.863          0          0          0          0
Speed: 2.7ms preprocess, 124.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving c:\Users\matrix\pruebayolo\proyecto_yolo\src\runs\segment\validation_affectation\predictions.json...
Results saved to c:\Users\matrix\pruebayolo\proyecto_yolo\src\runs\segment\validation_affectation


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de fuentes</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Para utilizar múltiples fuentes de datos al realizar predicciones con el modelo, se requiere que se ajuste el parámetro'source' a sus necesidades, tal como se indica en la siguiente tabla:</span>


| Fuentes          | Asignación                             | Tipo             | Notas                                                           |
| --------------- | ------------------------------------ | ----------------- | --------------------------------------------------------------- |
| `image`           | 'image.jpg'                          | str or Path       | Archivo que contiene una única imagen.                                              |
| `URL`             | 'https://ultralytics.com/images/bus.jpg' | str               | Dirección que especifica la ubicación de una imagen en la web.                                                 |
| `screenshot`      | 'screen'                             | str               | El sistema captura la imagen actualmente visible en la pantalla y la utiliza como entrada para el modelo.                                           |
| `PIL`             | Image.open('im.jpg')                 | PIL.Image         | Utilizado para cargar imágenes en formato HWC (altura, ancho, canales) con canales RGB (rojo, verde y azul) mediante la biblioteca Python Imaging Library (PIL).                                   |
| `OpenCV`          | cv2.imread('im.jpg')                 | np.ndarray        | Permite la lectura de una imagen desde un archivo en formato HWC con canales BGR (azul, verde, rojo) utilizando la biblioteca OpenCV, almacenando la imagen como un array de NumPy.                    |
| `numpy`           | np.zeros((640,1280,3))               | np.ndarray        | Genera un array de ceros con las dimensiones especificadas para un formato HWC con canales BGR, utilizando la biblioteca NumPy.                    |
| `torch`           | torch.zeros(16,3,320,640)            | torch.Tensor      | Crea un tensor de ceros con las dimensiones especificadas para un formato HWC con canales RGB, empleando el framework PyTorch.               |
| `CSV`             | 'sources.csv'                        | str or Path       | Archivo de texto que almacena las rutas a las imágenes que se procesarán.   |
| `video`          | 'video.mp4'                          | str or Path       | Proporciona acceso a un archivo de video único.                       |
| `directory`      | 'path/'                              | str or Path       | Directorio que contiene múltiples archivos de imagen.               |
| `glob`           | 'path/*.jpg'                         | str               | Permite acceder a varias imágenes en un directorio usando expresiones de coincidencia de patrones. |
| `YouTube`        | 'https://youtu.be/LNwODJXcvt4'       | str               | Facilita el acceso a videos desde la plataforma YouTube.                                         |
| `stream`         | 'rtsp://example.com/media.mp4'      | str               | Permite la conexión a flujos de video o audio en tiempo real mediante protocolos como RTSP, RTMP, TCP o IP, ya sea a través de internet o una red local. |
| `multi-stream`   | 'list.streams'                       | str or Path       | Se utiliza para transmitir varios flujos de medios simultáneamente, permitiendo el procesamiento y análisis paralelo de múltiples flujos de medios. |

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Formatos para las imágenes: </span>

| Image Suffixes | Reference                           |
| --------------- | ----------------------------------- |
| .bmp            | [Microsoft BMP File Format](https://docs.fileformat.com/es/image/bmp/)           |
| .dng            | [Adobe DNG](https://docs.fileformat.com/es/image/dng/)                           |
| .jpeg           | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .jpg            | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .mpo            | [Multi Picture Object](https://docs.fileformat.com/es/image/mpo/)                |
| .png            | [Portable Network Graphics](https://docs.fileformat.com/es/image/png/)           |
| .tif            | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .tiff           | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .webp           | [WebP](https://docs.fileformat.com/es/image/webp/)                                |
| .pfm            | [Portable FloatMap](https://docs.fileformat.com/font/pfm/)                   |


<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Formatos para los videos: </span>

| Video Suffixes | Reference                           |
| -------------- | ----------------------------------- |
| .asf           | [Advanced Systems Format](https://docs.fileformat.com/es/video/asf/)             |
| .avi           | [Audio Video Interleave](https://docs.fileformat.com/es/video/avi/)              |
| .gif           | [Graphics Interchange Format]()          |
| .m4v           | [MPEG-4 Part 14](https://docs.fileformat.com/es/video/m4v/)                      |
| .mkv           | [Matroska](https://docs.fileformat.com/es/video/mkv/)                            |
| .mov           | [QuickTime File Format](https://docs.fileformat.com/es/video/mov/)               |
| .mp4           | [MPEG-4](https://docs.fileformat.com/es/video/mp4/)          |
| .mpeg          | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .mpg           | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .ts            | [MPEG Transport Stream](https://docs.fileformat.com/es/video/ts/)               |
| .wmv           | [Windows Media Video](https://docs.fileformat.com/es/video/wmv/)                 |
| .webm          | [WebM Project](https://docs.fileformat.com/es/video/webm/)                        |


In [ ]:
def convert_path(path): 
    return os.path.normpath(path).replace(os.sep, '/')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Manipulación múltiples clases</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">PREDICCIÓN</span>


In [6]:
SOURCE= dir_prueba # Establezca el origen de datos que el modelo de detección utilizará para realizar predicciones. Configure el valor de este hiperparámetro según la tabla proporcionada anteriormente.
MAX_DET=3 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara predecir en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada durante la predicción en tareas de detección. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda utilizar los mismos valores utilizados durante el entrenamiento del modelo para mantener la coherencia en la inferencia.
CONF=0.5 # Establece el umbral de confianza durante el proceso de predicción en la tarea de detección. Se recomienda establecer el valor hiperparámetro entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión en la inferencia.
LINE_WIDTH= None # Determina el grosor en píxeles de los cuadros delimitadores que rodean los objetos detectados por el modelo. Puede establecer el grosor de la línea como un número entero en el que, a mayor valor, la línea será más gruesa, también puede utilizar como valor None para que el grosor se ajuste de forma automatizada, proporcionando una línea proporcional al tamaño de la imagen.
VISUALIZE=False # Determina si las características del modelo de detección deben mostrarse durante la predicción. Establecer esto en True permite que las características se muestren como mapas intermedias, lo que hace que el modelo sea más fácil de entender. Si se establece en False, no se mostrarán las características del modelo. 
IOU=0.7 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,7. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren la misma detección. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.
DEVICE='cpu' # Especifica el dispositivo de ejecución para la prueba de predicción en la operación de detección. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
VID_STRIDE=False # Controla la velocidad de los fotogramas durante el proceso de predicción en vídeos o secuencias de tiempo real. Al establecerlo en True el modelo se adapta a la velocidad de fotogramas especificada por la fuente de vídeo, procesando cada fotograma individualmente. Para desactivar esta función indique como valor False.
STREAM_BUFFER=False # Controla el almacenamiento en búfer de los fotogramas para la detección. Si es True, se almacenan todos los fotogramas para el procesamiento en tiempo real de vídeos o transmisiones en directo; si es False, devuelve el fotograma más reciente.
SAVE_FRAMES=False # Controla la captura y almacenamiento de los fotogramas predichos por el modelo de detección. Con True, se guardarán todos los fotogramas individuales predichos; con False, no se realizará el almacenamiento de los fotogramas.
AUGMENT=True # Aplica transformaciones a las imágenes de entrada, tales como giros, rotaciones, recortes y cambios de color, para diversificar los datos y mejorar la predicción en la detección. Establecer en True para activar la función, False para desactivar.
SAVE_CROP=False # Determina si se deben guardar imágenes recortadas con los resultados durante la predicción en la detección. Al establecerlo en "False", las imágenes recortadas no se guardarán, lo que reduce el tamaño del archivo. Con el valor "True", se guardarán las imágenes recortadas correspondientes a las áreas detectadas.
SHOW=False # Determina si se deben mostrar las imágenes o vídeos detectados durante la predicción. Al establecerlo en "True", permite la visualización de las predicciones en el mismo entorno, proporcionando una representación visual de los resultados. Si se establece en "False", las predicciones no se mostrarán. 
SAVE_TXT= False #
SAVE_CONF=True #
SAVE=True #

def thread_safe_predict(model, output_queue, id_class, conf_value, name_experiment, data, saved_verification, txt_verification, max_detection): #  Realiza una predicción segura utilizando un modelo YOLO.
    try:
        local_model = YOLO(model) # Inicializa el modelo YOLO.
        results = local_model.predict(source= data, save=saved_verification, classes=id_class, conf=conf_value, name=name_experiment, save_txt=txt_verification, max_det=max_detection, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save_conf=SAVE_CONF, retina_masks=True, agnostic_nms=True) # Se establecen los hiperpametros a utilizar en el proceso de predicción.
        output_queue.put((results)) # Pone los resultados en la cola.
        
    except Exception as e: # Atrapa cualquier mensaje de error. 
        print(f'Error when executing the prediction task: {e}')

def create_crops(results_car, dir): #  Crea recortes a partir de objetos detectados en una imagen.
    for attribute_car in results_car: # Recorre los resultados de la segmentación de carro.
        img_data = np.copy(attribute_car.orig_img) # Copia la imagen original.
        img_path = Path(attribute_car.path) 
        img_name = img_path.stem # Obtiene el nombre de la imagen por medio de la ruta.
        img_extension = img_path.suffix # Obtiene la extensión de la imagen.

        for ci, c in enumerate(attribute_car): # Itera sobre cada objeto detectado.
            b_mask = np.zeros(img_data.shape[:2], np.uint8) # Crea una máscara binaria del tamaño de la imagen original, inicializada a ceros.
            contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2) # Extrae el contorno del objeto, lo convierte a enteros y lo reconfigura para drawContours.
            _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED) # Dibuja el contorno en la máscara binaria, rellenando el área del objeto.
             
            mask3ch = cv2.cvtColor(b_mask, cv2.COLOR_GRAY2BGR) # Convierte la máscara binaria a 3 canales.
            isolated = cv2.bitwise_and(mask3ch, img_data) # Permite aislar el objeto de la imagen. Para fondo trasparente "isolated = np.dstack([img_data, b_mask])".
            
            x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32) # Obtiene las coordenadas de la caja delimitadora.
            iso_crop = isolated[y1:y2, x1:x2] # Recorta el objeto aislado.

            file_name = f'{img_name}[{ci +1}]{img_extension}'  # Se crea un nombre de archivo para el recorte.
            cv2.imwrite(os.path.join(dir, file_name), iso_crop) # Guardar el recorte en el directorio temporal.

def temp_controller(results_car): # Maneja el directorio temporal para los resultados del coche.
    dir_temp = tempfile.mkdtemp() # Se crea un directorio temporal, establezca entre los (prefix='', dir='') si desea controlar el nombre y la ruta donde se guardara el directorio temporal.
    create_crops(results_car, dir_temp)  # Crea recortes a partir de los resultados de los coches.
    return dir_temp# Devuelve la ruta del directorio temporal.

def delete_temp(temp_car): # Permite eliminar un directorio temporal.
    try: 
        shutil.rmtree(temp_car)  # Elimina un directorio temporal según la ruta especificada.
        print(f'Temporary directory {temp_car} successfully deleted!')
    except Exception as e: # Captura cualquier excepción e imprime un mensaje de error.
        print(f'Error deleting the directory: {e}')

output_queue = Queue() # Creación de una cola para comunicación segura con hilos.
try: 
    Thread(target=thread_safe_predict, args=(model_orig, output_queue, 2, 0.2, "predict_car", SOURCE, False, False, MAX_DET)).start() # Inicio de un nuevo hilo para la predicción del coche.
    results_car= output_queue.get() # Obtiene los resultados de la cola.
    dir_temp_car=temp_controller(results_car) # Manejo de un directorio temporal para los resultados del coche.
    print(dir_temp_car)
    try:
        Thread(target=thread_safe_predict, args=(model_afectation, output_queue, [0, 1], 0.39, "predict_affectation_seg_srcv5-seg", dir_temp_car, SAVE, SAVE_TXT, MAX_DET)).start()  # Inicia un nuevo hilo para la predicción de afectación, la cual se basara en los resultados de vehiculo.
        results_afectation= output_queue.get() # Obtiene los resultados de la cola.
        #delete_temp(dir_temp_car) # Eliminar el directorio temporal para los resultados de los coches.
    except Exception as e: # Captura de excepciones para la predicción de afectación.
        print(f"No vehicle images found: {e} ")
        
except Exception as e: # Captura de excepciones para la predicción de coches.
    print(f"Error in prediction by: {e} ") 


image 1/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\20170531_010133.jpg: 384x640 1 car, 892.9ms
image 2/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\5-320w-320w.jpg: 480x640 1 car, 1022.7ms
image 3/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\Car-Dent.png: 224x640 1 car, 544.0ms
image 4/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\Charlotte-Toyota-service-1-1024x683.jpg: 448x640 2 cars, 1045.1ms
image 5/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\GSDH75MG6FAM7FSG43RFQLN5R4.jpg: 448x640 1 car, 958.8ms
image 6/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\OIP (1).jpg: 480x640 (no detections), 885.5ms
image 7/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\OIP (2).jpeg: 640x640 (no detections), 1225.7ms
image 8/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\OIP (4).jpeg: 480x640 1 car, 898.6ms
image 9/25 c:\Users\matri

In [35]:
# Control de hiperparámetros constantes
IMGSZ= 640
DEVICE= 'CPU'
SAVE= True
BATCH= 16

class Tasks_YOLO:
    def __init__(self, selected_task, model, name='intento', data='sin/data', imgsz=IMGSZ, device= DEVICE, save= SAVE, batch= BATCH, **kwargs):
        self.selected_task = selected_task
        self.model = model
        #self.project = project # Carpeta global donde se almacenaran los experimentos
        self.name = name # Carpeta individual de cada experimento
        self.data = data
        self.imgsz = imgsz
        self.device = device
        self.save = save # Controla si desea guardar los resultados del experimento
        self.batch = batch
        self.custom_settings= kwargs

    def control_configuration(self):
        if self.selected_task == 'train':
            self.training_settings= {
            'task': 'segment',
            'epochs': 30,
            'patience': 30,
            'optimizer': 'auto',
            'workers': 1,
            'plots': True, 
            'verbose': False, 
            'rect': False, 
            'cos_lr': False, 
            'fraction': 1.0, 
            'exist_ok': False, 
            'overlap_mask': True, 
            'mask_ratio': 1
            }
            self._remove_parameters(self.training_settings)
            self.training()
            
        elif self.selected_task == 'val':
            self.validation_settings = {
            'save_json': True,
            'save_hybrid': True,
            'conf': 0.5,
            'max_det': 10,
            'plots': True,
            'rect': False,
            'iou': 0.6
            }
            self._remove_parameters(self.validation_settings)
            self.validation()
            
        else:
            self.prediction_settings = {
            'conf': 0.4,
            'max_det': 10,
            'save_txt': False,
            'line_width': None,
            'visualize': False,
            'iou': 0.6,
            'vid_stride': False,
            'stream_buffer': False,
            'save_frames': False,
            'augment': True,
            'save_crop': False,
            'show': False,
            'save_conf': True
            }
            self._remove_parameters(self.prediction_settings)
            self.prediction()
    
    def _remove_parameters(self, dict_configuration):
        keys_to_remove = [param for param in self.custom_settings.keys() if param not in dict_configuration]
        
        for invalid_parameter in keys_to_remove:
            del self.custom_settings[invalid_parameter]
        
        self._apply_defaults(dict_configuration)
            
    def _apply_defaults(self, dict_configuration):
    # Aplica los valores por defecto a los hiperparámetros que no han sido modificados por el usuario
        for param, default in dict_configuration.items(): 
            if param not in self.custom_settings:  # Verifica si el usuario modificó el parámetro de manera manual
                self.custom_settings[param] = default  # Asigna un valor por defecto al parámetro si no fue modificado
        
    def _loading_model(self):
        try:
            return YOLO(self.model)
        except Exception as e:
            print(f"Error when loading the model: {e}")
    
    def training(self):
        # Configuración personalizada de entrenamiento 
        try: 
            print('iniciando proceso de entrenamiento! ')
            print(f'parametros: {self.custom_settings}')   
            self.instance_model= self._loading_model()
            self.instance_model.train(data=self.data, name=self.name, device=self.device, imgsz=self.imgsz, batch= self.batch,**self.custom_settings)
        except Exception as e:
            print(f'Error in the training: {e}')
    
    def validation(self):
        try:
            print('iniciando proceso de validacion! ')
            print(f'parametros: {self.custom_settings}')
            self.instance_model= self._loading_model()
            self.instance_model.val(data= self.data, name=self.name, device= self.device, imgsz= self.imgsz, batch= self.batch, **self.custom_settings)
        except Exception as e:
            print(f'Error in the validation process: {e}')
    
    """def prediction(self):
        try:
            print('iniciando proceso de prediccion! ')
            print(f'parametros: {self.custom_settings}')
            self.instance_model= self._loading_model()
            results = self.instance_model.predict(source= self.data, name=self.name, device= 'cpu', imgsz= 640, classes=2, **self.custom_settings) #Error when executing the prediction task: 'NoneType' object is not subscriptable
            
            output_queue.put((results))
            
        except Exception as e:
            print(f'Error when executing the prediction task: {e}')
        
    def _create_crops(self, results_car, dir_temp= 'lala/'): #  Crea recortes a partir de objetos detectados en una imagen.
        print('crops ', results_car)
        print('directorio car', dir_temp)
        for attribute_car in results_car: # Recorre los resultados de la segmentación de carro.
            img_data = np.copy(attribute_car.orig_img) # Copia la imagen original.
            img_path = Path(attribute_car.path) 
            img_name = img_path.stem # Obtiene el nombre de la imagen por medio de la ruta.
            img_extension = img_path.suffix # Obtiene la extensión de la imagen.

            for ci, c in enumerate(attribute_car): # Itera sobre cada objeto detectado.
                b_mask = np.zeros(img_data.shape[:2], np.uint8) # Crea una máscara binaria del tamaño de la imagen original, inicializada a ceros.
                contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2) # Extrae el contorno del objeto, lo convierte a enteros y lo reconfigura para drawContours.
                _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED) # Dibuja el contorno en la máscara binaria, rellenando el área del objeto.
                
                mask3ch = cv2.cvtColor(b_mask, cv2.COLOR_GRAY2BGR) # Convierte la máscara binaria a 3 canales.
                isolated = cv2.bitwise_and(mask3ch, img_data) # Permite aislar el objeto de la imagen. Para fondo trasparente "isolated = np.dstack([img_data, b_mask])".
                
                x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32) # Obtiene las coordenadas de la caja delimitadora.
                iso_crop = isolated[y1:y2, x1:x2] # Recorta el objeto aislado.

                file_name = f'{img_name}[{ci +1}]{img_extension}'  # Se crea un nombre de archivo para el recorte.
                cv2.imwrite(os.path.join(dir_temp, file_name), iso_crop) # Guardar el recorte en el directorio temporal."""
                
    """def _temp_controller(self, results_car): # Maneja el directorio temporal para los resultados del coche.
        print('resultados', results_car)
        self._create_crops(results_car)
        dir_temp = tempfile.mkdtemp() # Se crea un directorio temporal, establezca entre los (prefix='', dir='') si desea controlar el nombre y la ruta donde se guardara el directorio temporal.
        self._create_crops(results_car, dir_temp)  # Crea recortes a partir de los resultados de los coches.
        return dir_temp# Devuelve la ruta del directorio temporal."""

    """def _delete_temp(self, temp_car): # Permite eliminar un directorio temporal.
        print('dir eliminado', temp_car)
        try: 
            shutil.rmtree(temp_car)  # Elimina un directorio temporal según la ruta especificada.
            print(f'Temporary directory {temp_car} successfully deleted!')
        except Exception as e: # Captura cualquier excepción e imprime un mensaje de error.
            print(f'Error deleting the directory: {e}')"""

def model_menu():
    while True:
        try:
            option_val = input('Qué tipo de modelo desea utilizar, indique "original" o "entrenado". Si desea cancelar el proceso, indique "salir": ').lower()
            if option_val == "salir":
                break
            if option_val == "original":
                return model_orig
            elif option_val == "entrenado":
                return model_afectation
            else:
                print('Character not valid, try again. ')
        except Exception as e:
            print(f'Error when performing the validation task: {e}')

# Menú
def task_selection():
    yolo_tasks= {'1':'train', '2': 'val', '3':'predict'} # Se establecen los formatos a los que se puede exportar los modelos YOLO
    while True:
        task = input(f'Seleccione el número de la tarea yolo que desea utilizar: {", ".join([f"{identificador}: {valor}" for identificador, valor in yolo_tasks.items()])}. Si desea cancelar el proceso, indique "salir":').lower()        
        if task == "salir": 
            break
         
        elif task in yolo_tasks: # Verifica que el identificador coincida con un formato
            model = model_menu()
            if task == '1' or task == '2': # Funciona correctamente
                test_object = Tasks_YOLO(yolo_tasks[task], model, 'tasks', data= data_train, conf=0.4)
                test_object.control_configuration()
            else:
                test_object_car = Tasks_YOLO(yolo_tasks[task], model, 'predic', data=dir_prueba, conf=0.2) # genera error
                test_object_car.control_configuration()
        else: 
            print('Task not found! ')

task_selection()

iniciando proceso de prediccion! 
parametros: {'conf': 0.2, 'max_det': 10, 'save_txt': False, 'line_width': None, 'visualize': False, 'iou': 0.6, 'vid_stride': False, 'stream_buffer': False, 'save_frames': False, 'augment': True, 'save_crop': False, 'show': False, 'save_conf': True}

image 1/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\20170531_010133.jpg: 384x640 (no detections), 480.0ms
image 2/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\5-320w-320w.jpg: 480x640 (no detections), 597.0ms
image 3/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\Car-Dent.png: 224x640 (no detections), 315.0ms
image 4/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\Charlotte-Toyota-service-1-1024x683.jpg: 448x640 (no detections), 548.1ms
image 5/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_affectation\GSDH75MG6FAM7FSG43RFQLN5R4.jpg: 448x640 (no detections), 540.5ms
image 6/25 c:\Users\matrix\pruebayolo\proyecto_yol

In [25]:
# Opción valor por defecto flexible
#(source= data, save=saved_verification, classes=id_class, conf=conf_value, name=name_experiment, save_txt=txt_verification, max_det=max_detection, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save_conf=SAVE_CONF, retina_masks=True, agnostic_nms=True)
class Prediccion:
    def __init__(self, model, source, name, imgsz=640, **kwargs): 
        self.name= name
        self.model = model # dirección del modelo a exportar
        self.source = source # nombre del formato de exportación seleccionado 
        self.imgsz = imgsz # tamaño de imagen global
        self.hyperparameter = kwargs # diccionario con el valor de hiperparámetros a modificar
        self.formats_config = { # Configuración establecida por defecto, en la forma -> Formato : {hiperparámetro = valor}
            'device': 'cpu',
            'conf': 0.4,
            'max_det': 10,
            'save_txt': False,
            'visualize': False,
            'iou': 0.6,
            'line_width': None,
            'vid_stride': False,
            'stream_buffer': False,
            'save_frames': False,
            'augment': True,
            'save_crop': False,
            'show': False,
            'save_conf': False,
            'retina_masks': True,
            'agnostic_nms': True
        }
        
        
    def remove_parameters(self):
        # Elimina los parámetros que no son compatibles con el formato de exportación seleccionado
        keys_to_remove = [param for param in list(self.hyperparameter.keys()) if param not in self.formats_config]
        #[param for param in self.custom_settings.keys() if param not in self.prediction_settings]
        print(keys_to_remove)
           
        for invalid_parameter in keys_to_remove:
            del self.hyperparameter[invalid_parameter]
        
        self.apply_defaults()
            
    def apply_defaults(self):
    # Aplica los valores por defecto a los hiperparámetros que no han sido modificados por el usuario
        for param, default in self.formats_config.items():
            if param not in self.hyperparameter:  # Verifica si el usuario modificó el parámetro de manera manual
                self.hyperparameter[param] = default  # Asigna un valor por defecto al parámetro si no fue modificado
           
    def prediccion(self, id_class):
        print(f'Configuration included: imgsz={self.imgsz}, {", ".join([f"{k}={v}" for k, v in self.hyperparameter.items()])}')
        #try:
        model_pred = YOLO(self.model) 
        print('Successful predict: ')
        return model_pred.predict(source=self.source, imgsz=self.imgsz, classes= id_class, **self.hyperparameter)

        #except Exception as e: 
         #   print(f'Error in the predict of the model: {e}')

# Menú
object_car = Prediccion(model=model_afectation, name='p_car', source=dir_prueba, conf=0.2)
object_car.remove_parameters()
results_car_a = object_car.prediccion([0,1])

[]
Configuration included: imgsz=640, conf=0.2, device=cpu, max_det=10, save_txt=False, visualize=False, iou=0.6, line_width=None, vid_stride=False, stream_buffer=False, save_frames=False, augment=True, save_crop=False, show=False, save_conf=False, retina_masks=True, agnostic_nms=True
Successful predict: 



TypeError: 'NoneType' object is not subscriptable

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Función de área para mascaras segmentadas</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Dataframe con las áreas totales</span>

In [27]:

def calculate_area(vertices): 
    total_mask_area = 0 # Se inicializa una variable acumuladora para almacenar la suma total del área de cada máscara por clase en una imagen.
    for point in vertices: # Recorre los puntos de contorno que se encuentran separados por medio de una lista para cada región segmentada.
        x_coordinate = point[:, 0] # Asigna todas las filas correspondientes a la primera columna como coordenada X del formato de masks.xy que entrega por medio de arreglos los vértices (x, y) en pixel.
        y_coordinate = point[:, 1] # Asigna todas las filas correspondientes a la segunda columna como coordenada Y.
        mask_area = np.round(0.5 * abs(np.dot(x_coordinate, np.roll(y_coordinate, 1)) - np.dot(y_coordinate, np.roll(x_coordinate, 1)))).astype(int) # Almacena el resultado del área por máscara, basándose en la fórmula Shoelace o lazada de Gauss. Utiliza la suma y la resta de los productos cruzados de las coordenadas de los vértices del polígono; el término np.roll(y, 1) y np.roll(x, 1) crean versiones desplazadas circularmente, lo cual es esencial para que np.dot() calcule el producto punto de los vértices del polígono. La diferencia entre los productos cruzados se multiplicará por 1/2 tomando el valor absoluto.
        total_mask_area += mask_area #  Acumula cada área individual para entregar un área total de afectación por clase.
    return total_mask_area 

def find_coincidence(filename): 
    # El nombre del archivo que contiene un número de variante entre corchetes.
    return filename.split('[')[0] + os.path.splitext(filename)[1] # Extrae el nombre original de la imagen eliminando el número de variante entre corchetes y conservando la extensión del archivo.

def search_for_mask_afectation(segmentation_results, image_instance): # Encuentra subimágenes comunes en los resultados de segmentación y obtiene el área de cada clase de afectación (abolladura y rayón) para cada subimagen.
    sub_image_results = []
    for attribute in segmentation_results: # Se itera sobre los resultados de la segmentación.
        if find_coincidence(os.path.basename(attribute.path)) == image_instance: # Para relacionar ambos formatos de imagen sin sobrescribir las posibles subimágenes con el mismo nombre que se puedan encontrar, se envía a la función "find_coincidente" la base del nombre por parte de los resultados de afectación (esta tendrá un formato nombre_en_común[1].extensión_común).
            # En los resultados de las máscaras cuando se manejan múltiples clases de objetos (hiperparámetro de predicción classes= []) en una imagen, es difícil referenciar las coordenadas que pertenecen a cada clase de objeto detectado, por lo tanto, es necesario separar dichos resultados de coordenadas. 
            dent_coordinates = [] # Inicializa una lista temporal para las coordenadas de abolladura.
            scratch_coordinates = [] # Inicializa una lista temporal para las coordenadas de rayón.
            tensor = attribute.boxes.cls # Almacena los valores del tensor, que hacen referencia al cuadro delimitador de cada máscara.
            for position, value in enumerate(tensor): # Itera sobre los valores del tensor y las posiciones correspondientes.
                if value == 0.: # Si el valor del tensor es 0, corresponde a una detección de abolladura.
                    dent_coordinates.append(attribute.masks.xy[position]) # Añade las coordenadas de la máscara a la lista de abolladuras.
                elif value == 1.:  # Asegura que solo maneje dos clases de afectación.
                    scratch_coordinates.append(attribute.masks.xy[position]) # Añade las coordenadas de la máscara a la lista de rayon.
            dent_area, scratch_area = calculate_area(dent_coordinates), calculate_area(scratch_coordinates) # Calcula el área para cada clase de afectación.
            sub_image_results.append((attribute.path, dent_area, scratch_area)) # Añade los resultados a la lista principal. 
    return sub_image_results if sub_image_results else [] # Retorna una lista de tuplas que contienen el nombre de la subimagen, el área de abolladura y el área del rayón, ej: [(ruta_variante, área_total_abolladura, área_total_rayón)].

def process_main_detection(afectation_results, car_results): # Función main que controla todo el proceso.
    dataframe_columns= ['imagen', 'equivalencia', 'area abolladura', 'area rayon', 'area carro', 'afectacion'] # Se definen las columnas del DataFrame.
    data=[] # Se inicializa una lista vacía para almacenar los resultados de cada imagen.
    
    for car in car_results: # Se recorren principalmente los resultados de carro, ya que es el primer filtro por el que pasa el modelo de segmentación, de esta manera en la salida se podrán ver los resultados para todo el conjunto de datos original.
        image_control = f'{os.path.basename(car.path)}' # Extrae el nombre base de la imagen de control.
 
        sub_image_data = search_for_mask_afectation(afectation_results, image_control) # Buscar máscaras en la imagen de control.
        if sub_image_data:  
            for index, segment in enumerate(sub_image_data): # Recorre cada resultado de subimagen encontrada para la imagen de control.
                associated_image, dent_area, scratch_area = segment # Extraer la imagen asociada, el área de abolladura y el área de rayón.
                
                if dent_area > 0 or scratch_area > 0: # Si existe un area de afectación, se calcula el área del coche y el daño total.
                    car_area = search_for_mask_car(car) # Recibirá una lista que contendrá los valores de área total para detecciones de vehículos; por ejemplo, [87916, 5504] indica que encontró dos vehículos para la imagen control.
                    car_area_id= car_area[index] # Para vincular el área del vehículo con cada subimagen, se selecciona por posición.
                    damage_car=vehicle_damage(car_area_id, dent_area, scratch_area) # Optiene el daño de afectación en el vehiculo. 
                else: # En caso de no encontrarse áreas de afectación, se agregarán resultados predeterminados.
                    car_area_id= 0
                    damage_car= 0
                   
                data.append([image_control, f'{os.path.basename(associated_image)}', dent_area, scratch_area, car_area_id, f'{damage_car} px^2']) # Se agregan los resultados obtenidos con el formato [nombre_imagen_control, nombre_imagen_equivalente, clase_1, clase_2, clase_3, daño_total_imagen] tomando una columna por clase con el fin de una visión más general.
        else:
            data.append([image_control, 'sin coincidencia', 0, 0, 0, '0px^2']) # Si no se encuentran máscaras se añaden valores por defecto.
            
    df= pd.DataFrame(data, columns=dataframe_columns)  # Se crea un DataFrame a partir de los datos.
    grouped= df.groupby("imagen", group_keys=True)[['equivalencia', 'area abolladura', 'area rayon', 'area carro', 'afectacion']].apply(lambda x: x) # Agrupa el DataFrame por imagen y aplicar la función lambda.
    return grouped # Retorna un DataFrame de pandas agrupado con los resultados procesados.

def search_for_mask_car(car):
    car_area_data= [] # Se inicializa una lista vacía para almacenar las áreas de los coches.
    for index, _ in enumerate(car.boxes.cls): # En los resultados del modelo YOLO no se puede conocer la cantidad de vehiculos detectados para una imagen. Por lo tanto, es importante guiarse con el tensor para saber cuántos automóviles han sido detectados y poder extraer cada resultado para un procesamiento único.
        car_masks= car.masks[index] # Obtiene la máscara para el resultado del coche actual.
        car_area_data.append(calculate_area(car_masks.xy))  # Calcula el área de la máscara y la añade a la lista.
    return car_area_data # Retorna una lista de las áreas de cada coche detectado por imagen.

def vehicle_damage(car_area, dent_area, scratch_area):
    if dent_area > 0 and scratch_area > 0: # Comprueba si ambas áreas de la clase de afectación son mayores que 0.
        total_affected_area = abs((dent_area + scratch_area) - car_area) # Calcula el área total afectada restando el área del coche a la suma de las áreas de abolladuras y rayón.
    else:
        total_affected_area = abs(dent_area - car_area) if dent_area > 0 else abs(scratch_area - car_area) if scratch_area > 0 else 0 # Si sólo una de las áreas de afectación es mayor que 0, usa esa área para restarla al área del vehiculo.
    return total_affected_area # Devuelve un float que representa el área total afectada del coche.

dataframe = process_main_detection(results_afectation, results_car)
style= dataframe.style.set_properties(**{'text-align': 'right'})
style

# Dataframe con las áreas individuales

In [28]:
def calculate_area(vertices): 
    individual_mask_area= [] # Se inicializa una lista vacia para guardar el área individual de cada objeto segmentado.
    if vertices: # Verifica si existe resultados de mascara.xy que proporciona los datos en formato pixel.
        for point in vertices: # Recorre los puntos de contorno que se encuentran separados por medio de una lista para cada región segmentada.
            x_coordinate = point[:, 0] # Asigna todas las filas correspondientes a la primera columna como coordenada X del formato de masks.xy que entrega por medio de arreglos los vértices (x, y) en pixel. 
            y_coordinate = point[:, 1] # Asigna todas las filas correspondientes a la segunda columna como coordenada Y.
            mask_area = np.round(0.5 * abs(np.dot(x_coordinate, np.roll(y_coordinate, 1)) - np.dot(y_coordinate, np.roll(x_coordinate, 1)))).astype(int) # Almacena el resultado del área por máscara, basándose en la fórmula Shoelace o lazada de Gauss. Utiliza la suma y la resta de los productos cruzados de las coordenadas de los vértices del polígono; el término np.roll(y, 1) y np.roll(x, 1) crean versiones desplazadas circularmente, lo cual es esencial para que np.dot() calcule el producto punto de los vértices del polígono. La diferencia entre los productos cruzados se multiplicará por 1/2 tomando el valor absoluto.
            individual_mask_area.append(mask_area) # Añade a la lista el resultado del área.
    else:
        return 0 # Cuando no exista resultados de mascara se retorna como valor 0, que indica no haber área.
    return individual_mask_area 

def find_coincidence(filename):
    # El nombre del archivo que contiene un número de variante entre corchetes.
    return filename.split('[')[0] + os.path.splitext(filename)[1] # Extrae el nombre original de la imagen eliminando el número de variante entre corchetes y conservando la extensión del archivo.

def search_for_mask_afectation(segmentation_results, image_instance): # Encuentra subimágenes comunes en los resultados de segmentación y obtiene el área de cada clase de afectación (abolladura y rayón) para cada subimagen.
    sub_image_results = []
    for attribute in segmentation_results: # Se itera sobre los resultados de la segmentación.
        if find_coincidence(os.path.basename(attribute.path)) == image_instance: # Para relacionar ambos formatos de imagen sin sobrescribir las posibles subimágenes con el mismo nombre que se puedan encontrar, se envía a la función "find_coincidente" la base del nombre por parte de los resultados de afectación (esta tendrá un formato nombre_en_común[1].extensión_común).
            # En los resultados de las máscaras cuando se manejan múltiples clases de objetos (hiperparámetro de predicción classes= []) en una imagen, es difícil referenciar las coordenadas que pertenecen a cada clase de objeto detectado, por lo tanto, es necesario separar dichos resultados de coordenadas. 
            dent_coordinates = [] # Inicializa una lista temporal para las coordenadas de abolladura.
            scratch_coordinates = [] # Inicializa una lista temporal para las coordenadas de rayón.
            tensor = attribute.boxes.cls # Almacena los valores del tensor, que hacen referencia al cuadro delimitador de cada máscara.
            for position, value in enumerate(tensor): # Itera sobre los valores del tensor y las posiciones correspondientes.
                if value == 0.: # Si el valor del tensor es 0, corresponde a una detección de abolladura.
                    dent_coordinates.append(attribute.masks.xy[position]) # Añade las coordenadas de la máscara a la lista de abolladuras.
                elif value == 1.:  # Asegura que solo maneje dos clases de afectación.
                    scratch_coordinates.append(attribute.masks.xy[position]) # Añade las coordenadas de la máscara a la lista de rayon.
            dent_area, scratch_area = calculate_area(dent_coordinates), calculate_area(scratch_coordinates) # Calcula el área para cada clase de afectación.
            sub_image_results.append((attribute.path, dent_area, scratch_area)) # Añade los resultados a la lista principal. 
    return sub_image_results if sub_image_results else [] # Retorna una lista de tuplas que contienen el nombre de la subimagen, las áreas de abolladura y las áreas de rayón, ej: [(ruta_variante, áreas_abolladura, áreas_rayón)].
    
def process_main_detection(afectation_class, car_results): # Función main que controla todo el proceso.
    columnas= ['imagen', 'equivalencia', 'clase mascara', 'area mascara', 'afectacion'] # Se definen las columnas del DataFrame.
    data=[] # Se inicializa una lista vacía para almacenar los resultados de cada imagen.
    
    for car in car_results: # Se recorren principalmente los resultados de carro, ya que es el primer filtro por el que pasa el modelo de segmentación, de esta manera en la salida se podrán ver los resultados para todo el conjunto de datos original.
        control_image = os.path.basename(car.path) # Extrae el nombre base de la imagen de control.
        
        sub_image_data = search_for_mask_afectation(afectation_class, control_image) # Buscar máscaras en la imagen de control.
        
        if sub_image_data: 
            for index, segment in enumerate(sub_image_data): # Recorre cada resultado de subimagen encontrada para la imagen de control.
                associated_image, dent_area, scratch_area = segment # Extraer la imagen asociada, el área de abolladura y el área de rayón.
                
                if dent_area is not None or scratch_area is not None: # Si existe un area de afectación, se calcula el área del coche y el daño total.
                    car_area = search_for_mask_car(car) # Recibirá una lista que contendrá los valores de área total para detecciones de vehículos; por ejemplo, [87916, 5504] indica que encontró dos vehículos para la imagen control. 
                    damage_image= vehicle_damage(car_area[index], dent_area, scratch_area) # Optiene el daño de afectación, vinculando el área del vehículo con cada subimagen.
                else: # En caso de no encontrarse áreas de afectación, se agregarán resultados predeterminados.
                    car_area = 0
                    damage_image= 0

                # Para añadir los resultados obtenidos se utiliza el formato: [nombre_imagen_control, nombre_imagen_equivalente, identificador_clase, areas_individuales, daño_total_imagen]con el fin de buscar en cada imagen los resultados de todas las clases manejadas y evitar errores de la función explode de pandas en las listas.
                data.append([control_image, f'{os.path.basename(associated_image)}', 'abolladura', dent_area, f'{damage_image} px^2'])
                data.append([control_image, f'{os.path.basename(associated_image)}', 'rayon', scratch_area, f'{damage_image} px^2']) 
                data.append([control_image, f'{os.path.basename(associated_image)}', 'carro', car_area[index], f'{damage_image} px^2'])
        else:  
            data.append([f'{os.path.basename(control_image)}', 'sin coincidencia', 'sin mascara', 0,  f'0 px^2']) # Si no se encuentran máscaras se añaden valores por defecto.
            
    orden_clases=['abolladura', 'rayon', 'carro', 'sin mascara'] # Esta lista se creó simplemente con el fin de mantener un orden en la visualización de las clases para el dataframe.
    df= pd.DataFrame(data, columns=columnas) # Se crea un DataFrame a partir de los datos.
    df['Clase'] = pd.Categorical(df['clase mascara'], categories=orden_clases, ordered=True) 
    grouped_columns=df.groupby(['imagen', 'equivalencia', 'afectacion', 'Clase'], group_keys=True, as_index=True, observed=False)[[ 'area mascara']].apply(lambda x : x) # Agrupa el DataFrame por imagen y aplicar la función lambda.
    grouped_columns_expanded = grouped_columns.explode('area mascara') # Expande el formato de lista inicial de la columna 'area mascara' en diferentes filas manteniendo su eje en la agrupación.
    return grouped_columns_expanded

def search_for_mask_car(car):
    car_area_data= [] # Se inicializa una lista vacía para almacenar las áreas de los coches.
    for index, _ in enumerate(car.boxes.cls): # En los resultados del modelo YOLO no se puede conocer la cantidad de vehiculos detectados para una imagen. Por lo tanto, es importante guiarse con el tensor para saber cuántos automóviles han sido detectados y poder extraer cada resultado para un procesamiento único.
        mascara= car.masks[index] # Obtiene la máscara para el resultado del coche actual.
        car_area_data.append(sum(calculate_area(mascara.xy))) # Calcula el área de la máscara y la añade a la lista.
    return car_area_data # Retorna una lista de las áreas de cada coche detectado por imagen.

def sum_if_list(area):
    return sum(area) if isinstance(area, list) else 0 # Si el parametro es una lista, retornara la suma del contenido de lo contrario devolvera un 0.

def vehicle_damage(car_area, dent_area, scratch_area):
    # Como el parametro 'dent_area y scratch area' son listas con formato [área total por objeto según clase], se requiere obtener el resultado total de cada clase. 
    sum_dent = sum_if_list(dent_area) # Recibe la suma total de áreas de abolladuras en una imagen.
    sum_scratch = sum_if_list(scratch_area) # Recibe la suma total de áreas de rayon en una imagen.
    if sum_dent > 0 and sum_scratch > 0: # Comprueba si ambas áreas de la clase de afectación son mayores que 0.
        total_affected_area = abs((sum_dent + sum_scratch) - car_area) # Calcula el área total afectada restando el área del coche a la suma de las áreas de abolladuras y rayón.
    else:
        total_affected_area= abs(car_area - sum_dent) if sum_dent > 0 else abs(car_area - sum_scratch) if sum_scratch > 0 else 0 # Si sólo una de las áreas es mayor que 0, usa esa área para restarla al área del vehiculo.
    return total_affected_area # Devuelve un float que representa el área total afectada del coche.

dataframe = process_main_detection(results_afectation, results_car)
# En ocasiones, los valores predeterminados no permiten visualizar todos los fotogramas de datos en el marco de datos. Por lo tanto, se establecen el máximo de columnas y filas como None para mostrar absolutamente todo.
pd.set_option('display.max_rows', None) # Establece el numero maximo de filas que se mostrarán del dataframe, por defecto se establece en 60.
pd.set_option('display.max_columns', None) # Establece el número maximo de columnas que se mostraran, por defecto son 20.
dataframe


area mascara
imagen                                             equivalencia                                       afectacion  Clase                      
20170531_010133.jpg                                20170531_010133[1].jpg                             0 px^2      abolladura  0             0
                                                                                                                  rayon       1             0
                                                                                                                  carro       2       1110288
5-320w-320w.jpg                                    5-320w-320w[1].jpg                                 47942 px^2  abolladura  3         14932
                                                                                                                  rayon       4             0
                                                                                                                  carro       5         62874
Car-Dent.png                                       Car-Dent[1].png                                    100542 px^2 abolladura  6         34156
                                                                                                                  rayon       7             0
                                                                                                                  carro       8        134698
Charlotte-Toyota-service-1-1024x683.jpg            Charlotte-Toyota-service-1-1024x683[1].jpg         571487 px^2 abolladura  9             0
                                                                                                                  rayon       10         9142
                                                                                                                              10          673
                                                                                                                              10         3576
                                                                                                                  carro       11       584878
                                                   Charlotte-Toyota-service-1-1024x683[2].jpg         22819 px^2  abolladura  12         6581
                                                                                                                  rayon       13            0
                                                                                                                  carro       14        29400
GSDH75MG6FAM7FSG43RFQLN5R4.jpg                     GSDH75MG6FAM7FSG43RFQLN5R4[1].jpg                  438478 px^2 abolladura  15        38058
                                                                                                                              15        41488
                                                                                                                  rayon       16            0
                                                                                                                  carro       17       518024
OIP (1).jpg                                        sin coincidencia                                   0 px^2      sin mascara 18            0
OIP (2).jpeg                                       sin coincidencia                                   0 px^2      sin mascara 19            0
OIP (4).jpeg                                       OIP (4)[1].jpeg                                    677092 px^2 abolladura  20            0
                                                                                                                  rayon       21          284
                                                                                                                  carro       22       677376
OIP(3).jpeg                                        OIP(3)[1].jpeg                                     0 px^2      abolladura  23            0
           

In [7]:
import cv2

from ultralytics import YOLO, solutions

model = YOLO("C:/Users/matrix/pruebayolo/proyecto_yolo/src/srcv5/model_segmentation/segmentation/training_affectation/weights/best.pt")
names = model.model.names

#cap = cv2.VideoCapture("path/to/video/file.mp4")
im0 = cv2.imread("C:/Users/matrix/pruebayolo/proyecto_yolo/src/assets_affectation/R7.png")
assert im0 is not None, "Error reading video file"
#w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
#video_writer = cv2.VideoWriter("distance_calculation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init distance-calculation obj
dist_obj = solutions.DistanceCalculation(names=names, view_img=True)

"""while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)
    im0 = dist_obj.start_process(im0, tracks)
    video_writer.write(im0)""" 

tracks = model.track(im0, persist=True, show=False)
im0 = dist_obj.start_process(im0, tracks)

#cap.release()
#video_writer.release()
#cv2.destroyAllWindows()

cv2.imshow('Imagen de salida', im0)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 288x640 4 abolladuras, 166.0ms
Speed: 4.0ms preprocess, 166.0ms inference, 6.0ms postprocess per image at shape (1, 3, 288, 640)


In [10]:
import cv2
from ultralytics import YOLO, solutions

# Cargar el modelo
model = YOLO("C:/Users/matrix/pruebayolo/proyecto_yolo/src/srcv5/model_segmentation/segmentation/training_affectation/weights/best.pt")
names = model.model.names

# Leer la imagen
im0 = cv2.imread("C:/Users/matrix/pruebayolo/proyecto_yolo/src/assets_affectation/Charlotte-Toyota-service-1-1024x683.jpg")
assert im0 is not None, "Error al leer la imagen"

# Inicializar el objeto para el cálculo de distancias
dist_obj = solutions.DistanceCalculation(names=names, view_img=True)

# Rastrear los objetos en la imagen
tracks = model.track(im0, persist=True, show=False)

# Calcular las distancias
im0 = dist_obj.start_process(im0, tracks)

# Guardar la imagen de salida
cv2.imshow('Imagen de salida', im0)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 448x640 15 rayons, 234.0ms
Speed: 4.0ms preprocess, 234.0ms inference, 35.0ms postprocess per image at shape (1, 3, 448, 640)
